# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [8]:
from cfdmod.use_cases.pressure.path_manager import CePathManager

path_manager = CePathManager(
    output_path="./output/pressure",
    config_path="./fixtures/tests/pressure/Ce_params.yaml",
    mesh_path="./fixtures/tests/pressure/galpao/galpao.normalized.lnas",
    cp_data_path="./fixtures/tests/pressure/data/cp_t.resampled.hdf"
)

Read post-processing Ce config file

In [9]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeConfig
import pathlib

post_proc_cfg = CeConfig.from_file(path_manager.config_path)

post_proc_cfg

{'pattern_1': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['sfc1'], exclude=['sfc1'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 20.0], surfaces=['sfc1'])}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'pattern_2': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 250.0], y_intervals=[0.0, 200.0], z_intervals=[0.0, 20.0]), no_zoning=[], exclude=[], exceptions={}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])}

Read simulation config file [Example]

In [10]:
from nassu.cfg.model import ConfigScheme

sim_cfg_path = pathlib.Path("../nassu/tests/validation/results/test_simulation/test_simulation/setup_nassu_case_v1.5.yaml")

sim_cfg = ConfigScheme.from_file(sim_cfg_path).load_sim_cfgs()[0]

sim_cfg

[2023-10-18 16:49:53.96] [INFO] Loaded from '..\nassu\tests\validation\results\test_simulation\test_simulation\setup_nassu_case_v1.5.yaml' 1 simulations ['test_simulation:000']


SimulationConfigs(sim_id=0, name='test_simulation', parent=None, n_steps=20000, save_path=WindowsPath('tests/validation/results/test_simulation'), report=RuntimeFrequencyConfigsEnd(start_step=0, end_step=0, frequency=1000), data=DataConfigs(instantaneous={'default': MacrsExportConfigs(interval=RuntimeFrequencyConfigsStartEnd(start_step=0, end_step=0, frequency=5000), area_export=AreaModel(start=(0.0, 0.0, 0.0), end=(1.0, 1.0, 1.0), is_abs=False), max_lvl=-1, macrs=['rho', 'u'])}, probes=ProbesConfigs(historic_series={'default': SeriesConfigs(macrs=['rho'], interval=RuntimeFrequencyLevelConfigsDefault(start_step=0, end_step=0, frequency=1, lvl=0), lines={}, csvs={}, points={'static_pressure': PointConfigs(pos=(197.0, 40.0, 15.0))}, bodies={'galpao': BodySeriesConfigs(body_name='galpao', normal_offset=0.0625, element_type='cell')})}, spectrum_analysis=SpectrumAnalysisConfigs(macrs=[], points={})), export_IBM_nodes=ExportIBMNodesConfigs(start_step=0, end_step=0, frequency=0), divergence=D

Normalize LNAS Mesh [Example]

In [11]:
from nassu.lnas import LagrangianFormat

body_cfg = sim_cfg.domain.bodies["galpao"]

original_mesh = LagrangianFormat.from_folder(body_cfg.lagrangian_path)

new_verts = original_mesh.geometry.vertices.copy()

new_verts[:, 0] -= new_verts[:, 0].min()
new_verts[:, 1] -= new_verts[:, 1].min()
new_verts[:, 2] -= new_verts[:, 2].min()

original_mesh.geometry.vertices = new_verts

original_mesh.to_file(body_cfg.lagrangian_path / f"{original_mesh.name}.normalized.lnas")

print(original_mesh.name, original_mesh.geometry.vertices.shape, original_mesh.geometry.triangles.shape)

original_mesh.geometry.vertices[:, 0].min(), original_mesh.geometry.vertices[:, 1].min(), original_mesh.geometry.vertices[:, 2].min()

galpao (1549, 3) (2915, 3)


(0.0, 0.0, 0.0)

Read Normalized LNAS Mesh

In [12]:
from nassu.lnas import LagrangianFormat

mesh = LagrangianFormat.from_file(path_manager.mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read Pressure Coefficient data

In [13]:
import pandas as pd

cp_data = pd.read_hdf(path_manager.cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)

Add region index to pressure coefficient data

In [14]:
from cfdmod.api.vtk.write_vtk import write_polydata, merge_polydata
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import process_surface

n_timesteps = cp_data_to_use["time_step"].unique().shape[0]

processed_polydata = []

for cfg_label, cfg in post_proc_cfg.items():
    logger.info(f"Processing {cfg_label} ...")
    for sfc in mesh.surfaces.keys():
        if sfc in cfg.zoning.exclude:
            logger.info(f"Surface {sfc} ignored!")  # Ignore surface
            continue

        logger.info(f"Processing surface {sfc} ...")

        processed_surface = process_surface(
            body_mesh=mesh,
            sfc_label=sfc,
            cfg=cfg,
            cp_data=cp_data_to_use,
            n_timesteps=n_timesteps,
        )
        
        processed_polydata.append(processed_surface.polydata)

        logger.info(f"Processed surface {sfc}")

    # Output 5: VTK
    merged_polydata = merge_polydata(processed_polydata)
    write_polydata(path_manager.get_vtp_path(mesh.name, cfg_label), merged_polydata)

    logger.info(f"Merged polydata for {cfg_label}")

[2023-10-18 13:49:54,082] [INFO] - cfdmod - Processing pattern_1 ... (2993601672.py:10)
[2023-10-18 13:49:54,083] [INFO] - cfdmod - Processing surface L1_xp ... (2993601672.py:16)
[2023-10-18 13:49:54,114] [INFO] - cfdmod - Processed surface L1_xp (2993601672.py:28)
[2023-10-18 13:49:54,114] [INFO] - cfdmod - Processing surface L2_yp ... (2993601672.py:16)
[2023-10-18 13:49:54,293] [INFO] - cfdmod - Processed surface L2_yp (2993601672.py:28)
[2023-10-18 13:49:54,294] [INFO] - cfdmod - Processing surface L3_zp_yp ... (2993601672.py:16)
[2023-10-18 13:49:54,365] [INFO] - cfdmod - Processed surface L3_zp_yp (2993601672.py:28)
[2023-10-18 13:49:54,366] [INFO] - cfdmod - Processing surface L4_zp_ym ... (2993601672.py:16)
[2023-10-18 13:49:54,427] [INFO] - cfdmod - Processed surface L4_zp_ym (2993601672.py:28)
[2023-10-18 13:49:54,427] [INFO] - cfdmod - Processing surface L5_ym ... (2993601672.py:16)
[2023-10-18 13:49:54,581] [INFO] - cfdmod - Processed surface L5_ym (2993601672.py:28)
[2023